In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")

# Load all datasets
airports = pd.read_csv("/kaggle/input/ua-flights-dataset/Airports Data.csv")
bags = pd.read_csv("/kaggle/input/ua-flights-dataset/BagLevelData.csv")
flights = pd.read_csv("/kaggle/input/ua-flights-dataset/Flight Level Data.csv")
pnr_flight = pd.read_csv("/kaggle/input/ua-flights-dataset/PNR Remark Level Data.csv")
pnr_remarks = pd.read_csv("/kaggle/input/ua-flights-dataset/PNRFlightLevelData.csv")

# Shape summary
print(f"Airports: {airports.shape}")
print(f"Bags: {bags.shape}")
print(f"Flights: {flights.shape}")
print(f"PNR Flight: {pnr_flight.shape}")
print(f"PNR Remarks: {pnr_remarks.shape}")

In [ ]:
datasets = {
    "Airports": airports,
    "Bags": bags,
    "Flights": flights,
    "PNR Flight": pnr_flight,
    "PNR Remarks": pnr_remarks
}

for name, df in datasets.items():
    print(f"\n{name} — Shape: {df.shape}")  # computing shape
    print(df.head(3)) # showing data head
    print(df.isnull().sum()) # missing value detection
    print("-" * 60) # detect separator

In [ ]:
# Convert date columns to datetime
date_cols = [
    'scheduled_departure_datetime_local', 'scheduled_arrival_datetime_local',
    'actual_departure_datetime_local', 'actual_arrival_datetime_local',
    'scheduled_departure_date_local'
]
for c in date_cols:
    flights[c] = pd.to_datetime(flights[c])

# Derived features
flights['departure_delay'] = (flights['actual_departure_datetime_local'] - flights['scheduled_departure_datetime_local']).dt.total_seconds() / 60
flights['arrival_delay'] = (flights['actual_arrival_datetime_local'] - flights['scheduled_arrival_datetime_local']).dt.total_seconds() / 60
flights['ground_time_gap'] = flights['scheduled_ground_time_minutes'] - flights['minimum_turn_minutes']

# Delay distributions
plt.figure(figsize=(8,5))
sns.histplot(flights['departure_delay'], bins=40, kde=True)
plt.title("Departure Delay Distribution")
plt.show()

plt.figure(figsize=(8,5))
sns.histplot(flights['arrival_delay'], bins=40, kde=True)
plt.title("Arrival Delay Distribution")
plt.show()

# Avg delays by fleet
delay_by_fleet = flights.groupby('fleet_type')[['departure_delay', 'arrival_delay']].mean().sort_values('departure_delay', ascending=False)
print(delay_by_fleet)

# departure_delay → Actual departure - Scheduled departure (minutes me)

# arrival_delay → Actual arrival - Scheduled arrival (minutes me)

# ground_time_gap → Scheduled ground time - Minimum turnaround time
#  distribution : Helps to see how many flights are early/on-time/late.

In [ ]:
# Aggregate bags per flight
bag_summary = bags.groupby(['company_id', 'flight_number', 'scheduled_departure_date_local']).agg(
    total_bags=('bag_tag_unique_number', 'count'),
    transfer_bags=('bag_type', lambda x: (x == 'Transfer').sum()),
    checked_bags=('bag_type', lambda x: (x == 'Checked').sum())
).reset_index()

bag_summary['transfer_bag_ratio'] = bag_summary['transfer_bags'] / (bag_summary['total_bags'] + 1e-5)

# Visualization
plt.figure(figsize=(8,5))
sns.histplot(bag_summary['transfer_bag_ratio'], bins=20)
plt.title("Transfer Bag Ratio Distribution")
plt.show()
# Ye ratio batata hai ki flight ke total bags me se kitne transfer bags hai.

In [ ]:
# Convert date columns
pnr_remarks['scheduled_departure_date_local'] = pd.to_datetime(pnr_remarks['scheduled_departure_date_local'])
pnr_remarks['pnr_creation_date'] = pd.to_datetime(pnr_remarks['pnr_creation_date'])

# Aggregate passenger-level info to flight level
pnr_summary = pnr_remarks.groupby(['company_id', 'flight_number', 'scheduled_departure_date_local']).agg(
    total_pax=('total_pax', 'sum'),
    child_count=('is_child', lambda x: (x == 'Y').sum()),
    stroller_users=('is_stroller_user', lambda x: (x == 'Y').sum()),
    lap_child_total=('lap_child_count', 'sum'),
    basic_economy_pax=('basic_economy_ind', 'sum')
).reset_index()

# Create ratios
pnr_summary['child_ratio'] = pnr_summary['child_count'] / (pnr_summary['total_pax'] + 1e-5)
pnr_summary['basic_econ_ratio'] = pnr_summary['basic_economy_pax'] / (pnr_summary['total_pax'] + 1e-5)

# Visualization
plt.figure(figsize=(8,5))
sns.histplot(pnr_summary['child_ratio'], bins=20)
plt.title("Child Passenger Ratio")
plt.show()
#Ye ratios later difficulty_score banane me help karte hain (e.g. zyada child passengers → zyada operational complexity)

In [ ]:
# Count SSR per flight
ssr_summary = pnr_flight.groupby(['flight_number']).agg(
    ssr_count=('special_service_request', 'count')
).reset_index()

plt.figure(figsize=(8,5))
sns.histplot(ssr_summary['ssr_count'], bins=20)
plt.title("Special Service Request (SSR) Count per Flight")
plt.show()
#Flights with high SSR counts (e.g. 10+ requests) →

#Require more coordination (staff, equipment, timing).

#Increase operational difficulty, hence contribute to higher difficulty_score later.

In [ ]:
# Ensure consistent dtypes before merging
date_cols = ['scheduled_departure_date_local']

for col in date_cols:
    # Convert all to datetime
    flights[col] = pd.to_datetime(flights[col], errors='coerce')
    pnr_summary[col] = pd.to_datetime(pnr_summary[col], errors='coerce')
    bag_summary[col] = pd.to_datetime(bag_summary[col], errors='coerce')

# Also ensure company_id and flight_number types align
flights['company_id'] = flights['company_id'].astype(str)
pnr_summary['company_id'] = pnr_summary['company_id'].astype(str)
bag_summary['company_id'] = bag_summary['company_id'].astype(str)

flights['flight_number'] = flights['flight_number'].astype(int)
pnr_summary['flight_number'] = pnr_summary['flight_number'].astype(int)
bag_summary['flight_number'] = bag_summary['flight_number'].astype(int)

In [ ]:
# Merge Flights + Passenger Summary
master = flights.merge(pnr_summary, on=['company_id', 'flight_number', 'scheduled_departure_date_local'], how='left')

# Merge with Bags Summary
master = master.merge(bag_summary, on=['company_id', 'flight_number', 'scheduled_departure_date_local'], how='left')

# Merge with SSR summary
master = master.merge(ssr_summary, on='flight_number', how='left')

# Fill NaNs
master.fillna(0, inplace=True)

# Derived combined ratios
master['load_ratio'] = master['total_pax'] / (master['total_seats'] + 1e-5)
master['baggage_per_pax'] = master['total_bags'] / (master['total_pax'] + 1e-5)
master['complexity_score'] = (
    0.3 * master['child_ratio'] +
    0.3 * master['transfer_bag_ratio'] +
    0.2 * master['basic_econ_ratio'] +
    0.2 * (master['ssr_count'] / (master['total_pax'] + 1e-5))
)
# finally merging individual dataset to whole , for analysis  and then filling null value and then calculating load_ratio , baggage_per_passesnger  and complexity score  using weighted mean

In [ ]:
key_features = [
    'departure_delay', 'arrival_delay', 'ground_time_gap',
    'load_ratio', 'baggage_per_pax', 'child_ratio',
    'basic_econ_ratio', 'transfer_bag_ratio', 'complexity_score'
]

corr = master[key_features].corr()

plt.figure(figsize=(10,6))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title("Feature Correlation Matrix")
plt.show()

# Quick insight: Which features correlate most with delays
delay_corr = corr['arrival_delay'].sort_values(ascending=False)
print("\nFeature Importance w.r.t Arrival Delay:\n", delay_corr)
# finding  correlation between columns that which factors affects more on flights delay

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Select features for modeling
model_features = [
    'load_ratio', 'baggage_per_pax', 'child_ratio', 'basic_econ_ratio',
    'transfer_bag_ratio', 'complexity_score', 'scheduled_ground_time_minutes', 'minimum_turn_minutes'
]
target = 'arrival_delay'

X = master[model_features]
y = master[target]

# Normalize
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
#Feature selection 

# Target selection 

# Normalization 

# Train/Test split

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

X =  master[model_features]
y = master['arrival_delay'].clip(-60, 300)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBRegressor(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("R2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print(y_pred)
# predicting average dealy in minutes for unseen flights data

In [ ]:
# Add predictions back to master (align indices properly)
master.loc[X_test.index, 'predicted_score'] = y_pred

In [ ]:
import shap

explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)

# Summary plot
shap.summary_plot(shap_values, X_test)

In [ ]:
# Rank flights per day based on predicted delay score
master['daily_rank'] = master.groupby('scheduled_departure_date_local')['predicted_score'] \
                             .rank(method='first', ascending=False)

# Normalize rank to percentile per day
master['rank_percentile'] = master.groupby('scheduled_departure_date_local')['daily_rank'] \
                                  .transform(lambda x: x / x.max())

# Classification based on percentile
def classify(percentile):
    if percentile <= 0.3:
        return 'Difficult'
    elif percentile <= 0.7:
        return 'Medium'
    else:
        return 'Easy'

master['class'] = master['rank_percentile'].apply(classify)

In [ ]:
# Features contributing to difficulty
features = ['ground_time_gap', 'load_ratio', 'transfer_bag_ratio', 'ssr_count']

# For each flight, find which feature has maximum contribution
master['top_feature'] = master[features].idxmax(axis=1)

In [ ]:
import shap

explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)

# Convert SHAP values to dataframe
shap_df = pd.DataFrame(shap_values.values, columns=X_test.columns)

# Find top contributing feature for each flight
master.loc[X_test.index, 'top_feature'] = shap_df.abs().idxmax(axis=1)

In [ ]:
print(master.columns)

In [ ]:
# Use arrival airport code as destination
# Aggregate by destination airport
agg = master.groupby('scheduled_arrival_station_code')['complexity_score'].mean().sort_values(ascending=False)
top_destinations = agg.head(10)
print("Top 10 Difficult Destinations based on Complexity Score:\n", top_destinations)

In [ ]:
# Aggregate by top feature using the correct score column
top_drivers = master.groupby('top_feature')['complexity_score'].mean().sort_values(ascending=False)
print("Top Contributing Features:\n", top_drivers)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(data=master, x='top_feature', order=master['top_feature'].value_counts().index)
plt.title("Top Contributing Feature Counts")
plt.show()

In [ ]:
master[['flight_number',
        'scheduled_departure_date_local',
        'complexity_score',   # Replace difficulty_score with complexity_score
        'daily_rank',
        'class',
        'top_feature']].to_csv("/kaggle/working/operational_insights(2).csv", index=False)